In [1]:
from collections import defaultdict
 
costs  = {'W': {'A': 61, 'B': 72, 'C': 45, 'D': 55, 'E': 66},
          'X': {'A': 69, 'B': 78, 'C': 60, 'D': 49, 'E': 56},
          'Y': {'A': 59, 'B': 66, 'C': 63, 'D': 61, 'E': 47}}
demand = {'A': 11, 'B': 12, 'C': 9, 'D': 10, 'E': 8}
cols = sorted(demand.keys())
supply = {'W': 15, 'X': 20, 'Y': 15}
res = dict((k, defaultdict(int)) for k in costs)
g = {}
for x in supply:
    g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
for x in demand:
    g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])
 
while g:
    d = {}
    for x in demand:
        d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x]) if len(g[x]) > 1 else costs[g[x][0]][x]
    s = {}
    for x in supply:
        s[x] = (costs[x][g[x][1]] - costs[x][g[x][0]]) if len(g[x]) > 1 else costs[x][g[x][0]]
    f = max(d, key=lambda n: d[n])
    t = max(s, key=lambda n: s[n])
    t, f = (f, g[f][0]) if d[f] > s[t] else (g[t][0], t)
    v = min(supply[f], demand[t])
    res[f][t] += v
    demand[t] -= v
    if demand[t] == 0:
        for k, n in supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del demand[t]
    supply[f] -= v
    if supply[f] == 0:
        for k, n in demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del supply[f]
 
for n in cols:
    print ("\t", n),
print
cost = 0
for g in sorted(costs):
    print (g, "\t"),
    for n in cols:
        y = res[g][n]
        if y != 0:
            print (y),
        cost += y * costs[g][n]
        print ("\t"),
    print
print ("\n\nTotal Cost = ", cost)

	 A
	 B
	 C
	 D
	 E
W 	
6
	
	
9
	
	
	
X 	
5
	
5
	
	
10
	
	
Y 	
	
7
	
	
	
8
	


Total Cost =  2834


### PuLP Model

In [2]:
from pulp import *

In [20]:
# Creates a list of all the supply nodes
Warehouses = ["A","B", "C"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 15,
        "B": 20,
         "C": 15}

# Creates a list of all demand nodes
Markets = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 11,
        "2": 12,
        "3": 9,
        "4": 10,
        "5": 8}

In [21]:
# Creates a list of costs of each transportation path
costs = [   #Markets
        #1 2 3 4 5
         [61,72,45,55,66],#A  Warehouses
        [69,78,60,49,56], #B
        [59,66,63,61,47] #C
         ]

In [22]:
# Creates the prob variable to contain the problem data
prob = LpProblem("Wood Distribution Problem",LpMinimize)

In [23]:
# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,m) for w in Warehouses for m in Markets]

In [24]:
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = LpVariable.dicts("Route",(Warehouses,Markets),0,None,LpInteger)

NameError: name 'w' is not defined

In [28]:
# The objective function is added to prob first
prob += lpSum([route_vars[w][m]*costs[w][m] for (w,m) in Routes.index]), "Sum of Transporting Costs"

TypeError: 'builtin_function_or_method' object is not iterable

In [29]:
# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += lpSum([route_vars[w][m] for m in Markets]) <= supply[w], "Sum of Products out of Warehouse %s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Markets:
    prob += lpSum([route_vars[w][m] for w in Warehouses]) >= demand[m], "Sum of Products into Markets %s"%m

NameError: name 'm' is not defined